<a href="https://colab.research.google.com/github/weilipan/MachineLearing/blob/main/1_2_%E9%81%BA%E6%BC%8F%E5%80%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1-2-1 偵測與刪除遺漏值
處理真實數據時常會遇到數據不完整或數據遺漏的情況，遺漏值除了容易造成分析結果的偏誤外，也難以直接交由機器學習模型來擬合。

In [ ]:
#下載寶可夢檔案
!wget pokemon.csv https://gist.githubusercontent.com/armgilles/194bcff35001e7eb53a2a8b441e8b2c6/raw/92200bc0a673d5ce2110aaad4544ed6c4010f687/pokemon.csv

--2023-03-30 00:34:03--  http://pokemon.csv/
Resolving pokemon.csv (pokemon.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘pokemon.csv’
--2023-03-30 00:34:03--  https://gist.githubusercontent.com/armgilles/194bcff35001e7eb53a2a8b441e8b2c6/raw/92200bc0a673d5ce2110aaad4544ed6c4010f687/pokemon.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44028 (43K) [text/plain]
Saving to: ‘pokemon.csv’

pokemon.csv         100%[===================>]  43.00K  --.-KB/s    in 0.005s  

2023-03-30 00:34:03 (8.06 MB/s) - ‘pokemon.csv’ saved [44028/44028]

FINISHED --2023-03-30 00:34:03--
Total wall clock time: 0.5s
Downloaded: 1 files, 43K in 0.005s (8.06 MB/s)


In [ ]:
# 透過pandas的info()顯示的一覽表
import numpy as np
import pandas as pd
df=pd.read_csv('pokemon.csv',)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


In [ ]:
# 